In [1]:
!pip install langchain-pinecone pinecone-client langchain-huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolv

In [1]:
pip install pinecone

In [2]:
!pip install -q gradio langchain-pinecone pinecone-client langchain-huggingface


In [3]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets sentencepiece peft
!pip -q install pypdf
!pip -q install sentence-transformers
!pip -q install langchain-community
!pip -q install langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.0 which is incompatible.


In [4]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=57f1288b0f86c1378bc2396d8626c64a362d0fdf505b3c7a237636e5f1b228fd
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [5]:
!pip install tokenizers

In [6]:
#!pip install xformers

Libraries

In [7]:
!pip install langchain-classic
!pip install langchain-huggingface


In [9]:
!pip install -q gradio

In [10]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone as LC_Pinecone
from langchain_classic.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [11]:
import nltk
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Passing URLs

In [12]:
URL=['https://www.evolvingdev.com/post/what-is-llama-2',
     'https://www.sciencedirect.com/science/article/pii/S266734522300024X',
     'https://www.sciencenewstoday.org/what-is-gemini-ai-the-future-of-multimodal-artificial-intelligence-explained',
     'https://www.cnet.com/tech/services-and-software/what-is-copilot-everything-you-need-to-know-about-microsofts-ai-tools/']

In [13]:
loader=UnstructuredURLLoader(urls=URL)
data=loader.load()

In [14]:
data

[Document(metadata={'source': 'https://www.evolvingdev.com/post/what-is-llama-2'}, page_content='Understanding Llama 2: A Guide to Meta\'s Open-source Language Model\n\nRichard Lawrence\n\nRichard Lawrence ·\n\nAugust 01, 2023· 20 min read\n\nTable of Contents\n\nIntroduction\n\nDeveloped by Meta, the company formerly known as Facebook, Llama 2 is an advanced AI model that is open-source and available for commercial use. With its potential to revolutionize various fields, it\'s essential to understand what Llama 2 is, its features, use cases, and how it\'s transforming the AI landscape.\n\nLlama 2, representing the next generation of large language models, has been fine-tuned with over 1 million human annotations and trained on over 2 trillion tokens. It outperforms its predecessors on numerous benchmarks, including reasoning and coding tests. However, it\'s not just the technical superiority of Llama 2 that sets it apart. Meta\'s decision to open-source this model has opened new pathw

In [15]:
len(data)

4

Splitting Extracted Data into Chunks

In [16]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [17]:
text_chunks=text_splitter.split_documents(data)

In [18]:
len(text_chunks)

39

In [19]:
text_chunks[9]

Document(metadata={'source': 'https://www.evolvingdev.com/post/what-is-llama-2'}, page_content="The OSI defines open-source software as programs whose licenses give users the freedom to run the software for any purpose, study how it works, adapt it, and redistribute copies, either verbatim or with modifications. The license should not discriminate against any person or group or restrict users from creating other software.\nHowever, despite Meta's claims, Llama 2's license seems to contain restrictions that contradict the OSI's definition. For instance, the license prohibits the use of Llama 2 to generate training data for other language models. It also includes clauses that restrict usage by companies with over 700 million monthly active users, effectively limiting the utilization to smaller businesses and researchers. From this perspective, Llama 2 cannot be considered genuinely open-source, as its license fails to assure the four essential freedoms stipulated by OSI.")

Downloading HuggingFaceEmbedding

In [20]:
embeddings=HuggingFaceEmbeddings()

/tmp/ipython-input-1837158031.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
/tmp/ipython-input-1837158031.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens),

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

Testing HF Embedding

In [22]:
a=embeddings.embed_query("Rahul")
len(a)

768

In [23]:
a

[0.032528843730688095,
 -0.022710710763931274,
 -0.021310240030288696,
 0.008791549131274223,
 -0.03780701756477356,
 -0.0005740030901506543,
 -0.04502520710229874,
 -0.06147684529423714,
 0.005728407762944698,
 0.0639132559299469,
 0.07376281172037125,
 0.04287856072187424,
 0.0046770828776061535,
 -0.013830282725393772,
 -0.005318112671375275,
 -0.03396114334464073,
 0.0006280160741880536,
 -0.009078082628548145,
 0.05204173922538757,
 0.0054017528891563416,
 -0.03191943094134331,
 0.03331369534134865,
 -0.009052437730133533,
 0.05877607688307762,
 0.009989834390580654,
 -0.007920531556010246,
 0.029046310111880302,
 0.027790062129497528,
 -0.031936392188072205,
 -0.040488213300704956,
 0.019586339592933655,
 -0.03743806481361389,
 0.037782106548547745,
 -0.01005878858268261,
 1.8035957509709988e-06,
 -0.015054981224238873,
 0.006868834141641855,
 -0.015232984907925129,
 0.009283540770411491,
 -0.0630633607506752,
 0.06224186718463898,
 0.01764119602739811,
 -0.004253130406141281,
 0

384 re-establishment

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_384 = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print(len(embeddings_384.embed_query("check")))  # must print 384


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

384


Converting Text Chunks into Embeddings and Creating a Knowledge Base

In [25]:
import os

PINECONE_API_KEY='pcsk_9uK9o_82Vdb2t6w3CUMMbNB2DZTgnp7a2b2K9kBkuE2QZTeMwMJPrrHDJdTeZPW73FX7W'
PINECONE_API_ENV='us-east-1'

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['PINECONE_ENVIRONMENT'] = PINECONE_API_ENV

In [30]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_4NjoRg_5pzWfhnfsmnP2s9kJtpUf4MGVzhivWNHJCr3dqZP8fyo4AA3cLgo5y5tviGc23j")
index = pc.Index("llama-384")

In [31]:
pc.list_indexes()


[
    {
        "name": "llama",
        "metric": "cosine",
        "host": "llama-6qnzu9c.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 768,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "llama-384",
        "metric": "cosine",
        "host": "llama-384-6qnzu9c.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 384,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [32]:
index_name='llama'

Establishing 384 Pinecone index

In [33]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create the index (only run once!)
pc.create_index(
    name="llama-384",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",        # or "gcp"
        region="us-east-1"  # pick a valid region
    )
)

print("Index created successfully!")


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin, access-control-request-method, access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '334b000049e874d9dc0e3e37a0e604da', 'date': 'Wed, 24 Dec 2025 03:58:35 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [35]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

PINECONE_API_KEY = "pcsk_9uK9o_82Vdb2t6w3CUMMbNB2DZTgnp7a2b2K9kBkuE2QZTeMwMJPrrHDJdTeZPW73FX7W"
INDEX_NAME = "llama"

# 1️⃣ Init Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# 2️⃣ Define embeddings FIRST
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L12-v2"  # 768-dim
)

# 3️⃣ Connect to existing index (STRING name only)
vectorstore = PineconeVectorStore.from_existing_index(
    INDEX_NAME,
    embeddings,
    text_key="text"
)

# 4️⃣ (Optional) Inspect index stats
index = pc.Index(INDEX_NAME)
print("✅ VectorStore connected successfully!")
print(index.describe_index_stats())


✅ VectorStore connected successfully!
{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 54}},
 'total_vector_count': 54,
 'vector_type': 'dense'}


Establishing new vectorstore

In [36]:
from langchain_pinecone import PineconeVectorStore

vectorstore_384 = PineconeVectorStore.from_existing_index(
    "llama-384",
    embeddings_384,
    text_key="text"
)


In [37]:
vectorstore_384.add_documents(text_chunks)


['489d37cd-b527-4fff-bfcd-94c44190ce4a',
 'ddf973c7-d06e-4a91-bec5-9459d858e6c7',
 '7cd50dd3-4ca7-4339-b028-783012494a2f',
 'a743e099-a83a-4757-8951-a2306f10a93a',
 '3d7a2c0b-6108-470c-9aeb-b4c8d6216231',
 'a0670a76-d441-4ad5-9a68-2dcc3eb601a4',
 'd7c541e0-6016-4353-abe0-34ef5f435e98',
 '344bb28c-5761-476b-b0ec-24a912514d1a',
 'c9891a17-99b2-48b3-abc0-c806c41243be',
 '7d6a98ea-654e-4ac2-a628-9c14f18f7f17',
 '6d3eec96-db8e-480c-bfbe-efdd294e7d07',
 'c85d0df2-c948-4250-bb45-e4eb1e74e2b2',
 '1c8bc765-eed6-437d-97f2-81c3ac8835c7',
 '959a2e67-6c78-4181-977d-6d4a87ac8927',
 '20d35ee2-52fe-4aa3-9621-f819cbb8235c',
 '82d37707-9cb6-4e0a-8f34-32abbf5669e8',
 '4e857f4a-f49d-49f1-a752-a80b1d0e1186',
 '33da890e-4182-4499-bba5-c6405001ad68',
 'c6638df8-40b6-402f-844d-b56d213f8aa9',
 '79e89d86-85ca-4bf2-94d1-8bbfc014ca83',
 '3d774b1e-bfdb-42f6-8415-7242970215da',
 'abeb5a04-51eb-486e-af0b-459e96170984',
 '9d5bf5f3-fb77-4082-a7ff-acd6e672432e',
 '87fa759b-f062-46e8-ada3-e8dd4c5e2703',
 '59663c63-9c3f-

Creating LLM Wrapper

In [ ]:
notebook_login()

In [44]:
model="daryl149/llama-2-7b-chat-hf"

In [45]:
tokenizer=AutoTokenizer.from_pretrained(model,
                                        use_auth_token=True,)
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True,
                                           load_in_8bit=True,
                                           )

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [46]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map="auto",
              max_new_tokens=512,
              do_sample=True,
              top_k=30,
              num_return_sequences=1,
              eos_token_id=tokenizer.eos_token_id)

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


In [47]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.2})

In [ ]:
response = llm.invoke("Please provide a concise summary of ChatGPT and Gemini")
print(response)


In [ ]:
response = llm.invoke("On what parameters does Iphone 13 differs from Iphone 15")
print(response)


In [48]:
from langchain_classic.chains import RetrievalQA
print("✅ RetrievalQA imported successfully!")

✅ RetrievalQA imported successfully!


In [54]:
query="How was Chatgpt used for Students and Research Works"

In [ ]:
response = llm.invoke(query)
print(response)


In [55]:
print(len(vectorstore_384 ._embedding.embed_query("dimension check")))

doc=vectorstore_384 .similarity_search(query,k=3)

384


In [56]:
from langchain_core.documents import Document


In [57]:
print(type(doc[0]))
print(doc[0].page_content)
print(doc[0].metadata)


<class 'langchain_core.documents.base.Document'>
Copilot can also appear inside your apps and work directly with your content. For example, instead of asking ChatGPT to write an email and then copying it into Outlook, Copilot lives inside Outlook and generates the draft using your email history and context. It saves time and reduces the number of steps to get things done.
As an AI assistant, it resembles how Google's Gemini works across its software products, but Copilot is more tightly embedded into productivity tools. Meta AI is also integrated across its social platforms, but is better for casual and creative use.
What can Microsoft Copilot do?
{'source': 'https://www.cnet.com/tech/services-and-software/what-is-copilot-everything-you-need-to-know-about-microsofts-ai-tools/'}


In [58]:
doc

[Document(id='695150a0-6e37-4921-b879-f1700903d425', metadata={'source': 'https://www.cnet.com/tech/services-and-software/what-is-copilot-everything-you-need-to-know-about-microsofts-ai-tools/'}, page_content="Copilot can also appear inside your apps and work directly with your content. For example, instead of asking ChatGPT to write an email and then copying it into Outlook, Copilot lives inside Outlook and generates the draft using your email history and context. It saves time and reduces the number of steps to get things done.\nAs an AI assistant, it resembles how Google's Gemini works across its software products, but Copilot is more tightly embedded into productivity tools. Meta AI is also integrated across its social platforms, but is better for casual and creative use.\nWhat can Microsoft Copilot do?"),
 Document(id='bba34a76-feee-411a-a3fe-26f922a782ec', metadata={'source': 'https://www.cnet.com/tech/services-and-software/what-is-copilot-everything-you-need-to-know-about-micros

In [ ]:
test_vec = embeddings.embed_query("test")
print(len(test_vec))


In [59]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore_384.as_retriever())


In [60]:
question="What are capabilities of ChatGPT"

In [61]:
qa.run(question)

/tmp/ipython-input-3005738683.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  qa.run(question)


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nCopilot can also appear inside your apps and work directly with your content. For example, instead of asking ChatGPT to write an email and then copying it into Outlook, Copilot lives inside Outlook and generates the draft using your email history and context. It saves time and reduces the number of steps to get things done.\nAs an AI assistant, it resembles how Google's Gemini works across its software products, but Copilot is more tightly embedded into productivity tools. Meta AI is also integrated across its social platforms, but is better for casual and creative use.\nWhat can Microsoft Copilot do?\n\nSee full bio\nBarbara Pazur\nApril 17, 2025 5:00 a.m. PT\n7 min read\nMicrosoft just celebrated its 50th anniversary. The tech giant's success story means it's now often referred to as one of the Big Tech Big Five or Magn

Prompt Template

In [83]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Use ONLY the following context from {URL}: {context}

Question: {input}

If the context doesn't contain relevant information, respond exactly:
"I can only answer questions about {URL}"

Answer:
""")

# Rebuild your qa chain with strict prompt
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [68]:
from langchain_classic.chains import RetrievalQA

# Create retriever
retriever = vectorstore_384.as_retriever(search_kwargs={"k": 3})  # limit to top 3 docs

# Setup QA without any memory
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# Ask your question
question = "Who founded ChatGPT"
result = qa({"query": question})

print(result['result'])  # Only uses retrieved docs


KeyboardInterrupt: 

In [119]:
import gradio as gr
from gradio.components import Textbox

def chat_with_url(question):
    result = qa.invoke({"query": question})
    return result["result"]

with gr.Blocks(title="URL Q&A Bot") as demo:
    gr.Markdown("# 🚀 URL Q&A Bot")
    gr.Markdown(f"Query your Pinecone chunks from {URL}!")

    with gr.Row():
        with gr.Column(scale=1):
            question = Textbox(
                label="Question",
                placeholder=f"Ask about {URL}...",
                lines=3,
                show_label=True
            )
        with gr.Column(scale=2):
            answer = Textbox(
                label="Answer",
                lines=10,
                max_lines=20,
                interactive=False  # Read-only answer box
            )

    # FIX: Clear question after submit + button backup
    btn = gr.Button("Submit", variant="primary")
    btn.click(chat_with_url, question, answer)
    question.submit(chat_with_url, question, answer).then(
        lambda: "", question, question  # Clear input after submit
    )

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1052: UserWarning: Expected 0 arguments for function <function <lambda> at 0x7d42fc728fe0>, received 1.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1060: UserWarning: Expected maximum 0 arguments for function <function <lambda> at 0x7d42fc728fe0>, received 1.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86f9e2a2552e46db98.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


AttributeError: module 'gradio' has no attribute 'blocks'